## DATA CLEAN

In [2]:
import re
import pandas as pd
import numpy as np
import datetime

In [4]:
'''importation du df'''
df = pd.read_csv(r'C:\Users\Qw\Desktop\projet_gaming\laurent\vgsales_complet.csv')



'''comparer les nom des jeux entre fichier de laurent et vg_sales (metacritic vs vg_sales)'''
samename=[]
for i, j in zip(df['Name'],df['gamename']):
    if i in j or j in i:
        samename.append(1)
    else:
        samename.append(0)
df['samename']=samename 




'''recupération des données : '''
    # extraire les données de la colonne userscorewrap pour corriger les erreurs de
    #la colonne ratings et nb_votes
y=[]
for i in df['userscorewrap']:
    x = re.findall(r'[0-9\.0-9]+',i)
    y.append(x)
df['userscorewrap2']=y
    # remplacer les listes vides ou avec une seul valeur par [0,0] afin 
    # d'éviter l'erreur de "length..."
for i in df['userscorewrap2']:
    if i==[]:
        i.append(0)
        i.append(0)
    if len(i)==1:
        i.clear()
        i.append(0)
        i.append(0)
    # extraire les données de la colonne metascorewrap pour corriger les erreurs de
    #la colonne metascore et nb_critic
n=[]
for i in df['metascorewrap']:
    x = re.findall(r'[0-9\.0-9]+',i)
    n.append(x)
df['metascorewrap2']=n
    # remplacer les listes vides ou avec une seul valeur par [0,0] afin d'éviter l'erreur de "length..."
for i in df['metascorewrap2']:
    if i==[]:
        i.append(0)
        i.append(0)
    if len(i)==1:
        i.clear()
        i.append(0)
        i.append(0)
    # recuperer les données de la colonnes metascorewrap2 et userscorewrap2
x=[]
y=[]
for i in df['userscorewrap2']:
    x.append(i[0])
    y.append(i[1])
df['ratings']=x
df['nb_votes']=y

x=[]
y=[]
for i in df['metascorewrap2']:
    x.append(i[0])
    y.append(i[1])
df['metascore']=x
df['nb_critic']=y

    # recuperer série (si série donc =1 sinon 0)
serieB=[]
def extract_series(x): 
    r = re.compile(r"Series:.*\n") 
    result=r.findall(x)
    if result!=[]:
        y=result[0]
        return(y[8:-2]) 
    else:
        return('0')
for i in df['description']:
    serieB.append(extract_series(i))
df['serieB']=serieB
df['serieB']=df['serieB'].apply(lambda x: '1' if x!='0' else x)
df['serieB']=df['serieB'].astype(int)

    # recuperer Awards (si awards donc =1 sinon 0)
Awards=[]
def extract_awards(x): 
    r = re.compile(r"Awards:.*\n") 
    result=r.findall(x)
    if result!=[]:
        y=result[0]
        return(y[8:-1])                
    else:
        return('0')
for i in df['description']:
    Awards.append(extract_awards(i))
df['Awards']=Awards
df['Awards']=df['Awards'].apply(lambda x: '1' if x!='0' else x)
df['Awards']=df['Awards'].astype(int)

    # recuperer Like_Percent
Like_Percent =[]
def extract_percentage(x):
    r = re.compile(r"\n\d+% liked this video game")
    result=r.findall(x)
    if result!=[]:
        y=result[0] 
        return(y[1:3]) 
    else:
        return('0')
for i in df['description']:
    Like_Percent.append(extract_percentage(i))
df['Like_Percent']=Like_Percent
df['Like_Percent']=df['Like_Percent'].str.replace(r'%','').astype(int)
z=[]
for i in df['Like_Percent']:
    z.append(i/100)
df['Like_Percent']=z



'''supprimer les jeux non compatible avec vg_salles'''
df.drop(df[df['samename']==0].index, inplace=True)



'''nettoyage : '''
    # remplacer les nan de la colonne year par 0 et la convertir en int
df['Year']=df['Year'].replace(np.nan, 0)
df['Year']=df['Year'].astype(int)
    # remplacer les nan de la colonne Publisher par not found
df['Publisher']=df['Publisher'].replace(np.nan, 'unknown')
    # convertir les types des colonnes:
df['ratings']=df['ratings'].astype(float)
df['nb_votes']=df['nb_votes'].astype(int)
df['metascore']=df['metascore'].astype(int)
df['nb_critic']=df['nb_critic'].astype(int)


        
'''supprimer les colonnes dont on a plus besoin:'''
df.drop(['top','metascorewrap','userscorewrap','nbvotes','nbratings','description'
        ,'userscorewrap2','metascorewrap2'], axis=1, inplace=True)


'''renomer les colonnes'''
df = df.rename(columns = {'ratings':'userNote','nb_votes':'userNbVote'
                          ,'metascore':'metaNote','nb_critic':'metaNbVote'})


# quantiles (1 = les jeux mauvais...)
df['metaNoteQ']=pd.cut(df['metaNote'], bins=[0,25,50,75,100], labels=[1,2,3,4])
df['metaNoteQ']=df['metaNoteQ'].fillna(1)
df['metaNoteQ']=df['metaNoteQ'].astype(int)

'''reorganisation df '''
df = df[['Rank', 'Name','developper','Platform', 'Year','Genre', 'Publisher','NA_Sales',
      'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales','gamename','metaNote','metaNbVote'
         ,'userNote','userNbVote','samename','Like_Percent','metaNoteQ','serieB']] 

df.to_csv(r'C:\Users\Qw\Desktop\projet_gaming\vgsales_Zak.csv',index = False)

df.head()

,Rank,Name,developper,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,...,Global_Sales,gamename,metaNote,metaNbVote,userNote,userNbVote,samename,Like_Percent,metaNoteQ,serieB
0,1,Wii Sports,Nintendo,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,...,82.74,Wii Sports,76,51,8.1,453,1,0.96,4,1
1,2,Super Mario Bros.,Nintendo,NES,1985,Platform,Nintendo,29.08,3.58,6.81,...,40.24,Classic NES Series: Super Mario Bros.,84,14,8.5,98,1,0.92,4,1
2,3,Mario Kart Wii,Nintendo,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,...,35.82,Mario Kart Wii,82,73,8.4,1047,1,0.96,4,0
3,4,Wii Sports Resort,Nintendo,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,...,33.00,Wii Sports Resort,80,73,8.2,293,1,0.97,4,1
5,6,Tetris,Nintendo,GB,1989,Puzzle,Nintendo,23.20,2.26,4.22,...,30.26,Tetris 99,83,27,8.2,258,1,0.00,4,0
